In [91]:
import sagemaker
import boto3
from sagemaker.pytorch import PyTorch
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()
account_id = boto3.client('sts').get_caller_identity().get('Account')
region_name = boto3.session.Session().region_name
images_s3uri = 's3://{0}/lora/images/'.format(bucket)

In [92]:
##注意：lora train的图像目录下需要metadata.jsonl，其中file_name字段必须##########3
imgs="https://d1xkebsgyt7kzd.cloudfront.net/R_1.jpg,https://d1xkebsgyt7kzd.cloudfront.net/R_2.jpg,https://d1xkebsgyt7kzd.cloudfront.net/R_3.jpg,https://d1xkebsgyt7kzd.cloudfront.net/R_4.jpg,https://d1xkebsgyt7kzd.cloudfront.net/R_5.jpg"
for image in imgs.split(","):
    !wget $image
!mv ./*.jpg ./images/
!aws s3 cp images $images_s3uri --recursive

--2023-02-28 13:23:05--  https://d1xkebsgyt7kzd.cloudfront.net/R_1.jpg
Resolving d1xkebsgyt7kzd.cloudfront.net (d1xkebsgyt7kzd.cloudfront.net)... 13.33.100.75, 13.33.100.121, 13.33.100.45, ...
Connecting to d1xkebsgyt7kzd.cloudfront.net (d1xkebsgyt7kzd.cloudfront.net)|13.33.100.75|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38281 (37K) [image/jpeg]
Saving to: ‘R_1.jpg’

R_1.jpg             100%[===================>]  37.38K  --.-KB/s    in 0s      

2023-02-28 13:23:05 (293 MB/s) - ‘R_1.jpg’ saved [38281/38281]

--2023-02-28 13:23:05--  https://d1xkebsgyt7kzd.cloudfront.net/R_2.jpg
Resolving d1xkebsgyt7kzd.cloudfront.net (d1xkebsgyt7kzd.cloudfront.net)... 13.33.100.108, 13.33.100.45, 13.33.100.121, ...
Connecting to d1xkebsgyt7kzd.cloudfront.net (d1xkebsgyt7kzd.cloudfront.net)|13.33.100.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33919 (33K) [image/jpeg]
Saving to: ‘R_2.jpg’

R_2.jpg             100%[===================>] 

In [93]:
#image_uri = '{0}.dkr.ecr.{1}.amazonaws.com/all-in-one-ai-stable-diffusion-webui-training'.format(account_id, region_name)
image_uri = '687912291502.dkr.ecr.ap-southeast-1.amazonaws.com/lora-finetuning:latest'
models_s3uri = 's3://{}/stable-diffusion/models/{}/{}'.format(bucket,'lora','model.tar.gz')

In [11]:
import json
def json_encode_hyperparameters(hyperparameters):
    for (k, v) in hyperparameters.items():
        print(k, v)
    
    return {k: json.dumps(v) for (k, v) in hyperparameters.items()}



image_uri = '687912291502.dkr.ecr.ap-southeast-1.amazonaws.com/lora-finetuning:latest'
instance_type = 'ml.g4dn.xlarge'

s3_model_output_location='s3://{}/{}/{}'.format(bucket, 'lora', 'trained_models')
base_model_name="runwayml/stable-diffusion-v1-5"
output_dir="/opt/ml/model/"
#HUB_MODEL_ID="pokemon-lora"
dataset_name="lambdalabs/pokemon-blip-captions"



hyperparameters = {
                    'model_name':'aws-trained-lora-model',
                    'mixed_precision':'fp16',
                    'pretrained_model_name_or_path': base_model_name,
                    'dataset_name':dataset_name,
                    #'train_data_dir':'/opt/ml/input/data/images/',
                    'dataloader_num_workers':8,
                    'max_grad_norm':1,
                    'output_dir':output_dir,
                    'checkpointing_steps':1000,
                    'validation_prompt':'Totoro',
                    'seed':1338
                    #'manul_upload_model_path':s3_model_output_location,
                    'resolution':512,
                    'train_batch_size':1,
                    'gradient_accumulation_steps':4,
                    'learning_rate':2e-06,
                    'lr_scheduler':'cosine',
                    'lr_warmup_steps':0,
                    'max_train_steps':1000
}

hyperparameters = json_encode_hyperparameters(hyperparameters)

from sagemaker.estimator import Estimator
#inputs = {
#    'images': images_s3uri,
#    'models': models_s3uri
#}

estimator = Estimator(
    role = role,
    instance_count=1,
    instance_type = instance_type,
    image_uri = image_uri,
    hyperparameters = hyperparameters,
    environment = environment
)
estimator.fit()



train-args {"train_dreambooth_settings": {"db_create_new_db_model": true, "db_new_model_name": "aws-db-new-model", "db_new_model_src": "768-v-ema.ckpt", "db_new_model_scheduler": "ddim", "db_create_from_hub": false, "db_new_model_url": "", "db_new_model_token": "", "db_new_model_extract_ema": false, "db_model_name": "", "db_lora_model_name": "", "db_lora_weight": 1, "db_lora_txt_weight": 1, "db_train_imagic_only": false, "db_use_subdir": false, "db_custom_model_name": "", "db_train_wizard_person": false, "db_train_wizard_object": true, "db_performance_wizard": true}}
train-task dreambooth
sd-models-s3uri s3://sagemaker-ap-southeast-1-687912291502/stable-diffusion/models/
db-models-s3uri s3://sagemaker-ap-southeast-1-687912291502/stable-diffusion/dreambooth/
ckpt /opt/ml/input/data/models/768-v-ema.ckpt
dreambooth-config-id dreambooth_train_config
api-endpoint noapi


In [ ]:
lora_model_data = estimator.model_data
print("Model artifact saved at:\n", dreambooth_model_data)

In [97]:
!aws s3 ls s3://sagemaker-ap-southeast-1-687912291502/dreambooth-finetuning-v3-2023-02-28-13-23-42-549/output/model.tar.gz

2023-02-28 14:34:09 2640400606 model.tar.gz
